In [1]:
#%run input/Format.ipynb
import ROOT as root
from array import array
root.gErrorIgnoreLevel = root.kFatal
import pandas as pd
%jsroot on

Welcome to JupyROOT 6.30/06


In [ ]:
iOption0 = 3 # piminus, electron

In [ ]:
colors=[1,2,4,root.kGreen+2,root.kMagenta,root.kOrange+4,root.kGray,root.kCyan]
central_bins = [0,95,0,20,20,40,40,60,60,100]
N_centr = int(len(central_bins)/2)
pt_boarders = [0.2,0.3,0.4,0.5,0.6,0.7,0.9,1.1,1.4,1.9,2.5,3.5,5.0]
N_pt = len(pt_boarders)-1
file_path="/home/yoren/bnl/PHENIX/ee/AuAu/"
file_names=["my-50M_piminus_embed_single_new_04-10_v0.root","my-50M_electron_embed_single_new_04-10_v0.root","my-100M_piminus_single_hagedorn_04-5_v0.root"\
            ,"my-100M_piminus_embed_single_new_04-10_v0"]
file_name=file_names[iOption0]
outfilenames= ['pionminus','electron','pionminus1']#output
filename=outfilenames[iOption0]
part_ids=[9,3,9,9]
part_id = part_ids[iOption0]
part_name = ["e^{+}","e^{-}"]
N_start, N_end = 0e6, 100e6

In [4]:
root.gSystem.Load("../AnaTrain/Run14AuAuLeptonComby/MyEvent_C.so")

0

In [5]:
root.gROOT.ProcessLine(
f"TFile *input = new TFile(\"{file_path+file_name}\", \"READ\");\
TTree *T = (TTree *)input->Get(\"tree\");\
MyDileptonAnalysis::MyEvent *myevent = 0;\
TBranch *br = T->GetBranch(\"MyEvent\");\
br->SetAddress(&myevent);\
MyDileptonAnalysis::MyEventContainer *event_container = new MyDileptonAnalysis::MyEventContainer();\
event_container->InitEvent();\
event_container->GetHistsFromFile(\"../../ee_QA/AnaTrain/Run14AuAuLeptonComby/field_map.root\");\
"
); 
from ROOT import myevent, br, event_container

File opened at ../../ee_QA/AnaTrain/Run14AuAuLeptonComby/field_map.root


mcid: 2 - positron, 3 - electron, 8 - pion+, 9 - pion minus

In [6]:
geant3dict = {
  2: "positron",
  3: "electron",
  5: "+muon",
  6: "-muon",
  8: "+pion",
  9: "-pion",
  11: "+kaon",
  12: "-kaon",
  14: "proton",
  15: "antiproton"
}

In [7]:
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0
for ievent in range(N_real_ev):
    if good_event_counter>60: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.6 or myevent.GetEntry(ipart).GetN0()>=0:
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    print(f"Nevent: {ievent}, centrality: {myevent.GetCentrality()}, x: {myevent.GetPreciseX()}, Ntracks: {myevent.GetNtrack()}") 
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        print("pt: ", round(myevent.GetEntry(ipart).GetPtPrime(),3),", e/p: ",\
              round(myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),3), ", n0: ",\
              myevent.GetEntry(ipart).GetN0(),", disp: ", round(myevent.GetEntry(ipart).GetDisp(),2), ", chi2/npe0: ",\
              round(myevent.GetEntry(ipart).GetChi2()/(myevent.GetEntry(ipart).GetNpe0()+1e-4),2), ", prob: ",\
              round(myevent.GetEntry(ipart).GetProb(),3), ", id: ",  geant3dict[myevent.GetEntry(ipart).GetMcId()])
    for ipart in range(myevent.GetNgentrack()):
        print(myevent.GetGenTrack(ipart).GetPt())

99748438
Nevent: 27, centrality: 1.0, x: 0.34185999631881714, Ntracks: 1
pt:  0.657 , e/p:  0.819 , n0:  -9999 , disp:  -9999.0 , chi2/npe0:  1.0 , prob:  0.2 , id:  -pion
0.6536231637001038
Nevent: 33, centrality: 83.0, x: 0.33152198791503906, Ntracks: 1
pt:  0.592 , e/p:  1.285 , n0:  -9999 , disp:  -9999.0 , chi2/npe0:  1.0 , prob:  0.314 , id:  -pion
0.5902518033981323
Nevent: 41, centrality: 16.0, x: 0.3329020142555237, Ntracks: 1
pt:  0.589 , e/p:  0.618 , n0:  -9999 , disp:  -9999.0 , chi2/npe0:  1.0 , prob:  0.816 , id:  -pion
0.5869799256324768
Nevent: 43, centrality: 22.0, x: 0.3311469852924347, Ntracks: 1
pt:  0.601 , e/p:  0.906 , n0:  -9999 , disp:  -9999.0 , chi2/npe0:  1.0 , prob:  0.251 , id:  -pion
0.5945274829864502
Nevent: 49, centrality: 45.0, x: 0.34380099177360535, Ntracks: 1
pt:  0.415 , e/p:  1.009 , n0:  -9999 , disp:  -9999.0 , chi2/npe0:  1.0 , prob:  0.934 , id:  -pion
0.4200933277606964
Nevent: 54, centrality: 9.0, x: 0.3470749855041504, Ntracks: 1
pt:  0.8

In [8]:
data_array = []
N_real_ev = br.GetEntries()
print(N_real_ev)
good_event_counter = 0

for ievent in range(int(N_start), min(int(N_end),N_real_ev)):
    if (ievent - N_start) % 1e5 == 0: print("N of events:", ievent - N_start)
    if good_event_counter>100000000: break
    myevent.ClearEvent()
    br.GetEntry(ievent)
    event_container.SetEvent(myevent)
    if not myevent.GetNtrack(): continue
    do_continue = False
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.5 or myevent.GetEntry(ipart).GetN0()>=0):
            do_continue = True
    if not do_continue:
        continue
    good_event_counter+=1
    event_container.correct_beam_offset()
    for ipart in range(myevent.GetNtrack()):
        if myevent.GetEntry(ipart).GetMcId()==part_id and (myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot()>0.5 or myevent.GetEntry(ipart).GetN0()>=0):
            data_array.append([int(myevent.GetCentrality()),myevent.GetEntry(ipart).GetZDC(),myevent.GetEntry(ipart).GetPtPrime(),\
                           myevent.GetEntry(ipart).GetEcore()/myevent.GetEntry(ipart).GetPtot(),myevent.GetEntry(ipart).GetN0(),\
                           myevent.GetEntry(ipart).GetDisp(),min(myevent.GetEntry(ipart).GetChi2(), myevent.GetEntry(ipart).GetChi2()/(myevent.GetEntry(ipart).GetNpe0()+1e-6)),\
                            myevent.GetEntry(ipart).GetNpe0(),\
                           myevent.GetEntry(ipart).GetProb(),myevent.GetEntry(ipart).GetDisp()**2])


data = pd.DataFrame(data_array, columns=('centrality','zdc','pt', 'e/p', 'n0', 'disp', 'chi2/npe0', 'npe0', 'prob', 'disp2'))

99748438
N of events: 0.0
N of events: 100000.0
N of events: 200000.0
N of events: 300000.0
N of events: 400000.0
N of events: 500000.0
N of events: 600000.0
N of events: 700000.0
N of events: 800000.0
N of events: 900000.0
N of events: 1000000.0
N of events: 1100000.0
N of events: 1200000.0
N of events: 1300000.0
N of events: 1400000.0
N of events: 1500000.0
N of events: 1600000.0
N of events: 1700000.0
N of events: 1800000.0
N of events: 1900000.0
N of events: 2000000.0
N of events: 2100000.0
N of events: 2200000.0
N of events: 2300000.0
N of events: 2400000.0
N of events: 2500000.0
N of events: 2600000.0
N of events: 2700000.0
N of events: 2800000.0
N of events: 2900000.0
N of events: 3000000.0
N of events: 3100000.0
N of events: 3200000.0
N of events: 3300000.0
N of events: 3400000.0
N of events: 3500000.0
N of events: 3600000.0
N of events: 3700000.0
N of events: 3800000.0
N of events: 3900000.0
N of events: 4000000.0
N of events: 4100000.0
N of events: 4200000.0
N of events: 4300

In [9]:
data.head()

,centrality,zdc,pt,e/p,n0,disp,chi2/npe0,npe0,prob,disp2
0,64,77.812500,1.275886,0.578487,-9999,-9999.0,-9999.0,-9999,0.970612,99980001.0
1,20,11.117188,0.843373,0.536502,-9999,-9999.0,-9999.0,-9999,0.000055,99980001.0
2,1,5.742188,0.656944,0.818837,-9999,-9999.0,-9999.0,-9999,0.200266,99980001.0
3,83,15.234375,0.592385,1.285034,-9999,-9999.0,-9999.0,-9999,0.313758,99980001.0
4,76,40.593750,0.869461,0.598230,-9999,-9999.0,-9999.0,-9999,0.947787,99980001.0


In [10]:
print(data.size, data.shape, good_event_counter)

93106100 (9310610, 10) 9286038


In [11]:
filepath = '/home/yoren/bnl/PHENIX/ee/ML/'+filename
compression_options = dict(method='zip', archive_name=f'{filename}.csv')

In [12]:
data.to_csv(f'{filepath}.zip', compression=compression_options, index=False)